# Demo: Implementing Row Level Security (M04\_L04\_Demo 2)

**<span style="color:black;mso-color-alt:windowtext">Objective<i>:&nbsp; </i></span>** <span style="color:black;
mso-color-alt:windowtext">The goal of this demo is to demonstrate how to implement Row Level Security in Azure Synapse SQL Pool.&nbsp;&nbsp;</span>

In [1]:
--Create Sales Table
CREATE TABLE Sales  
    (  
    OrderID int,  
    SalesRep sysname,  
    Product varchar(10),  
    Qty int  
    );


Commands completed successfully.

Total execution time: 00:00:00.599

In [2]:
--Insert some records
INSERT INTO Sales VALUES (1, 'SalesA', 'Valve', 5);
INSERT INTO Sales VALUES (2, 'SalesA', 'Wheel', 2);
INSERT INTO Sales VALUES (3, 'SalesA', 'Valve', 4);
INSERT INTO Sales VALUES (4, 'SalesB', 'Bracket', 2);
INSERT INTO Sales VALUES (5, 'SalesB', 'Wheel', 5);
INSERT INTO Sales VALUES (6, 'SalesB', 'Seat', 5);

-- View the 6 rows in the table  
SELECT * FROM Sales;


(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(6 rows affected)

Total execution time: 00:00:00.560

OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


In [3]:
--Create some users for future use
CREATE USER Manager WITHOUT LOGIN;  
CREATE USER SalesA WITHOUT LOGIN;  
CREATE USER SalesB WITHOUT LOGIN;


Commands completed successfully.

Total execution time: 00:00:00.184

In [4]:
--Grant Select on the tables
GRANT SELECT ON Sales TO Manager;  
GRANT SELECT ON Sales TO SalesA;  
GRANT SELECT ON Sales TO SalesB;


Commands completed successfully.

Total execution time: 00:00:00.204

**Run** the following query to create the security function. The function returns 1 when a row in the SalesRep column is the same as the user executing the query (@SalesRep = USER\_NAME()) or if the user executing the query is the Manager user (USER\_NAME() = 'Manager'):

In [5]:
CREATE SCHEMA Security;  
GO  
  
CREATE FUNCTION Security.fn_securitypredicate(@SalesRep AS sysname)  
    RETURNS TABLE  
WITH SCHEMABINDING  
AS  
    RETURN SELECT 1 AS fn_securitypredicate_result
WHERE @SalesRep = USER_NAME() OR USER_NAME() = 'Manager';


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.438

**Run** the following query to create a security policy adding the function as a filter predicate:

In [6]:
CREATE SECURITY POLICY SalesFilter  
ADD FILTER PREDICATE Security.fn_securitypredicate(SalesRep)
ON dbo.Sales  
WITH (STATE = ON);


Commands completed successfully.

Total execution time: 00:00:00.113

Allow SELECT permissions to the fn\_securitypredicate function

In [7]:
GRANT SELECT ON security.fn_securitypredicate TO Manager;  
GRANT SELECT ON security.fn_securitypredicate TO SalesA;  
GRANT SELECT ON security.fn_securitypredicate TO SalesB;


Commands completed successfully.

Total execution time: 00:00:00.156

1. **Run** following query to test the filtering predicate by selected from Sales table as each user.

In [8]:
EXECUTE AS USER = 'SalesA';  
SELECT * FROM Sales;
REVERT;  
  
EXECUTE AS USER = 'SalesB';  
SELECT * FROM Sales;
REVERT;  
  
EXECUTE AS USER = 'Manager';  
SELECT * FROM Sales;
REVERT;


(3 rows affected)

(3 rows affected)

(6 rows affected)

Total execution time: 00:00:01.375

OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2


OrderID,SalesRep,Product,Qty
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


\*\* The Manager should see all six rows. The Sales1 and Sales2 users should only see their own sales.

  

Disable the policy and re-run SELECT query.

In [9]:
ALTER SECURITY POLICY SalesFilter  
WITH (STATE = OFF);




Commands completed successfully.

Total execution time: 00:00:00.093

In [10]:
EXECUTE AS USER = 'SalesA';  
SELECT * FROM Sales;
REVERT;  
  
EXECUTE AS USER = 'SalesB';  
SELECT * FROM Sales;
REVERT;  
  
EXECUTE AS USER = 'Manager';  
SELECT * FROM Sales;
REVERT;

(6 rows affected)

(6 rows affected)

(6 rows affected)

Total execution time: 00:00:00.591

OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


OrderID,SalesRep,Product,Qty
1,SalesA,Valve,5
3,SalesA,Valve,4
2,SalesA,Wheel,2
6,SalesB,Seat,5
4,SalesB,Bracket,2
5,SalesB,Wheel,5


\*\* Now Sales1 and Sales2 users can see all six rows.

In [11]:
--Clean up objects
DROP USER SalesA;
DROP USER SalesB;
DROP USER Manager;

DROP SECURITY POLICY SalesFilter;
DROP TABLE Sales;
DROP FUNCTION Security.fn_securitypredicate;
DROP SCHEMA Security;


Commands completed successfully.

Total execution time: 00:00:00.828